# Import Libraries

In [1]:
import pandas as pd
import requests
from io import BytesIO
from collections import defaultdict

# Retrieve Data

In [2]:
restaurant_url = 'https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json'
country_codes_url = 'https://github.com/Papagoat/brain-assessment/blob/main/Country-Code.xlsx?raw=true'

restaurant_response = requests.get(restaurant_url)
restaurants_data = restaurant_response.json()

country_codes_response = requests.get(country_codes_url)
country_codes_df = pd.read_excel(BytesIO(country_codes_response.content))
country_codes_df

,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia
5,148,New Zealand
6,162,Phillipines
7,166,Qatar
8,184,Singapore
9,189,South Africa


# Data Extraction

## Question 1: Restaurants data

In [6]:
# List to store restuarant data
restaurants_list = []

# Extract the required fields from each restaurant
for restaurants in restaurants_data:
    for restaurant_entry in restaurants['restaurants']:
        restaurant = restaurant_entry['restaurant']

        # Check if there are any events for the restaurant
        if 'zomato_events' in restaurant and restaurant['zomato_events']:
            for event_wrapper in restaurant['zomato_events']:
                # Access the 'event' key inside the 'zomato_events'
                event = event_wrapper.get('event', {})

                if 'photos' in event and event['photos']:
                    photo_info = event['photos'][0].get('photo', {})
                    photo_url = photo_info.get('url', 'No Photo')
                    thumb_url = photo_info.get('thumb_url', 'No Thumbnail')
                else:
                    photo_url = 'No Photo'
                    thumb_url = 'No Thumbnail'
                
                restaurant_info = {
                    'Restaurant Id': restaurant['R']['res_id'],
                    'Restaurant Name': restaurant['name'],
                    'Country Id': restaurant['location']['country_id'],
                    'City': restaurant['location']['city'],
                    'User Rating Votes': restaurant['user_rating']['votes'],
                    'User Aggregate Rating': float(restaurant['user_rating']['aggregate_rating']),
                    "Rating Text": restaurant["user_rating"]["rating_text"],
                    'Cuisines': restaurant['cuisines'],
                    'Event Id': event.get('event_id', "NA"),
                    'Event Title': event.get('title', "NA"),
                    'Event Start Date': event.get('start_date', "NA"),
                    'Event End Date': event.get('end_date', "NA"),
                    'Event Start Time': event.get('start_time', "NA"),
                    'Event End Time': event.get('end_time', "NA"),
                    'Photo URL': photo_url,
                    'Thumbnail URL': thumb_url
                }

                # Append the restaurant and event information to the list
                restaurants_list.append(restaurant_info)
        else:
            # If no events, just include the restaurant information without event data
            restaurant_info = {
                'Restaurant Id': restaurant['R']['res_id'],
                'Restaurant Name': restaurant['name'],
                'Country Id': restaurant['location']['country_id'],
                'City': restaurant['location']['city'],
                'User Rating Votes': restaurant['user_rating']['votes'],
                'User Aggregate Rating': float(restaurant['user_rating']['aggregate_rating']),
                "Rating Text": restaurant["user_rating"]["rating_text"],
                'Cuisines': restaurant['cuisines'],
                'Event Id': "NA",
                'Event Title': "NA",
                'Event Start Date': "NA",
                'Event End Date': "NA",
                'Event Start Time': "NA",
                'Event End Time': "NA",
                'Photo URL': "NA",
                'Thumbnail URL': "NA"
            }

            restaurants_list.append(restaurant_info)

restaurants_df = pd.DataFrame(restaurants_list)
restaurants_df

,Restaurant Id,Restaurant Name,Country Id,City,User Rating Votes,User Aggregate Rating,Rating Text,Cuisines,Event Id,Event Title,Event Start Date,Event End Date,Event Start Time,Event End Time,Photo URL,Thumbnail URL
0,18649486,The Drunken Botanist,1,Gurgaon,4765,4.4,Very Good,"Continental, Italian, North Indian, Chinese",322331,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28,20:00:00,23:59:59,https://b.zmtcdn.com/data/zomato_events/photos...,https://b.zmtcdn.com/data/zomato_events/photos...
1,308322,Hauz Khas Social,1,New Delhi,13627,4.6,Excellent,"Continental, American, Asian, North Indian, Ch...",332812,Live 20/20 Match Screenings,2019-03-29,2019-05-23,09:00:00,01:35:00,https://b.zmtcdn.com/data/zomato_events/photos...,https://b.zmtcdn.com/data/zomato_events/photos...
2,18856789,AIR- An Ivory Region,1,New Delhi,1819,4.1,Very Good,"North Indian, Chinese, Continental, Asian",336644,Dhol Bhangra Night,2019-04-10,2019-04-11,21:00:00,00:00:00,https://b.zmtcdn.com/data/zomato_events/photos...,https://b.zmtcdn.com/data/zomato_events/photos...
3,307374,AMA Cafe,1,New Delhi,3252,4.4,Very Good,"Cafe, Juices",NA,NA,NA,NA,NA,NA,NA,NA
4,18238278,Tamasha,1,New Delhi,8112,4.4,Very Good,"Finger Food, North Indian, Continental, Italian",NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,7900186,Mexico,148,Hamilton,100,4.2,Very Good,Mexican,NA,NA,NA,NA,NA,NA,NA,NA
1396,7900182,Thai House Express,148,Hamilton,51,4.0,Very Good,Thai,NA,NA,NA,NA,NA,NA,NA,NA
1397,7900023,Indian Star,148,Hamilton,63,3.8,Good,"North Indian, Indian",NA,NA,NA,NA,NA,NA,NA,NA
1398,7900490,Zealong Tea Estate,148,Hamilton,55,4.7,Excellent,"European, Asian, Desserts",NA,NA,NA,NA,NA,NA,NA,NA


In [9]:
merged_df = pd.merge(restaurants_df, country_codes_df, how='left', left_on='Country Id', right_on='Country Code')
merged_df = merged_df[[
    'Restaurant Id', 
    'Restaurant Name', 
    'Country', 
    'City', 
    'User Rating Votes', 
    'User Aggregate Rating',
    'Cuisines'
]].drop_duplicates()
merged_df.to_csv('restaurants.csv', index=False)

## Question 2: Restuarant Events in month of April 2019
Assumption: 
1. The requirement specifies that events in April 2019 should be extracted. I interpret this to mean any event that either started or ended in April 2019. This allows me to capture all events that were active during the month, even if they began before April or ended after Aril
2. Only the relevant columns required by the task are included in the final CSV. Any additional data not specified in the requirements is omitted.

In [11]:
restaurants_df['Event Start Date'] = pd.to_datetime(restaurants_df['Event Start Date'], errors='coerce')
restaurants_df['Event End Date'] = pd.to_datetime(restaurants_df['Event End Date'], errors='coerce')

# Filter for events that happened in April 2019
april_2019_events = restaurants_df[
    ((restaurants_df['Event Start Date'].dt.month == 4) & (restaurants_df['Event Start Date'].dt.year == 2019)) |
    ((restaurants_df['Event End Date'].dt.month == 4) & (restaurants_df['Event End Date'].dt.year == 2019))
]

april_2019_events_filtered = april_2019_events[[
    'Event Id', 
    'Restaurant Id', 
    'Restaurant Name', 
    'Photo URL', 
    'Event Title', 
    'Event Start Date', 
    'Event End Date'
]].fillna("NA")
april_2019_events_filtered.to_csv('restaurant_events.csv', index=False)

C:\Users\aloys\AppData\Local\Temp\ipykernel_1500\3100879666.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].fillna("NA")
